<a href="https://colab.research.google.com/github/cvphelps/DeepLearningImplementations/blob/master/XGBoost_wandb_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Here's an example XGBoost project that classifies images of skin cancer.

Start by creating a free W&B account to track your training:

https://www.wandb.com/

In [7]:
# Install Weights & Biases to track training. First create an account at wandb.com
!pip install wandb -q
!wandb login
# Download the dermatology dataset
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/dermatology/dermatology.data

You can find your API keys in your browser here: https://app.wandb.ai/authorize
Paste an API key from your profile: 23b4991182751590ec0e6cc80e92144f080f3a2d
Appending key for api.wandb.ai to your netrc file: /root/.netrc
Successfully logged in to Weights & Biases!
--2019-08-03 01:20:27--  https://archive.ics.uci.edu/ml/machine-learning-databases/dermatology/dermatology.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25964 (25K) [application/x-httpd-php]
Saving to: ‘dermatology.data.4’

dermatology.data.4  100%[===================>]  25.36K  --.-KB/s    in 0.03s   

2019-08-03 01:20:27 (830 KB/s) - ‘dermatology.data.4’ saved [25964/25964]



In [9]:
# modified from https://github.com/dmlc/xgboost/blob/master/demo/multiclass_classification/train.py

%%wandb

import wandb
import numpy as np
import xgboost as xgb

wandb.init(project="xgboost-dermatology")

# label need to be 0 to num_class -1
data = np.loadtxt('./dermatology.data', delimiter=',',
        converters={33: lambda x:int(x == '?'), 34: lambda x:int(x) - 1})
sz = data.shape

train = data[:int(sz[0] * 0.7), :]
test = data[int(sz[0] * 0.7):, :]

train_X = train[:, :33]
train_Y = train[:, 34]

test_X = test[:, :33]
test_Y = test[:, 34]

xg_train = xgb.DMatrix(train_X, label=train_Y)
xg_test = xgb.DMatrix(test_X, label=test_Y)
# setup parameters for xgboost
param = {}
# use softmax multi-class classification
param['objective'] = 'multi:softmax'
# scale weight of positive examples
param['eta'] = 0.1
param['max_depth'] = 6
param['silent'] = 1
param['nthread'] = 4
param['num_class'] = 6
wandb.config.update(param)

watchlist = [(xg_train, 'train'), (xg_test, 'test')]
num_round = 5
bst = xgb.train(param, xg_train, num_round, watchlist, callbacks=[wandb.xgboost.wandb_callback()])
# get prediction
pred = bst.predict(xg_test)
error_rate = np.sum(pred != test_Y) / test_Y.shape[0]
print('Test error using softmax = {}'.format(error_rate))
wandb.summary['Error Rate'] = error_rate

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:614: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase memory consumption")


[0]	train-merror:0.011719	test-merror:0.127273
[1]	train-merror:0.015625	test-merror:0.127273
[2]	train-merror:0.011719	test-merror:0.109091
[3]	train-merror:0.007812	test-merror:0.081818
[4]	train-merror:0.007812	test-merror:0.090909
Test error using softmax = 0.09090909090909091
